In [ ]:
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_43_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_46112-46311_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_46112-46311_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_46591-46911_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_72_1_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_72_2_csv.zip


In [303]:
import pandas as pd 
import numpy as np
dir=["./denue_00_46111_csv/conjunto_de_datos/denue_inegi_46111_.csv", \
    "./denue_00_46112-46311_csv/conjunto_de_datos/denue_inegi_46112-46311_.csv", \
    "./denue_00_46321-46531_csv/conjunto_de_datos/denue_inegi_46321-46531_.csv", \
    "./denue_00_46591-46911_csv/conjunto_de_datos/denue_inegi_46591-46911_.csv", \
    "./denue_00_72_1_csv/conjunto_de_datos/denue_inegi_72_1.csv", \
    "./denue_00_72_2_csv/conjunto_de_datos/denue_inegi_72_2.csv"
    ]

cliente=pd.DataFrame([
[423,'Rio Lerma 65',19.43061268916689,-99.16507358911448],
[632,'Zacatecas 24',19.4166999785185,-99.1556043737739],
[702,'Oklahoma 84',19.391890374793046,-99.17759500279307],
[1021,'Av. Universidad 707',19.375386845013207,-99.16212560238631],
[937,'Felix Parra 117',19.366311887545983,-99.18504078433399]],
columns=['Spot','Dirección','latitud','longitud'])

In [304]:
def extraer_pizzeria(dir):
    df=pd.read_csv(dir,sep=',', encoding='ISO-8859–1',decimal='.',low_memory=False) 
    Pizzerias=pd.DataFrame([])
    Pizzerias['id']=df.id
    Pizzerias['nom_estab']=df.nom_estab
    Pizzerias['per_ocu']=df.per_ocu
    Pizzerias['latitud']=df.latitud
    Pizzerias['longitud']=df.longitud
    Pizzerias['Manzana']=df.nom_vial+'/'+df.nom_v_e_1+'/'+df.nom_v_e_2+'/'+df.nom_v_e_3
    Pizzerias['Numero']=df.numero_ext
    Pizzerias['Y_alta']=df.fecha_alta.str.slice(0,4)
    del df
    a=np.where((Pizzerias.nom_estab.str.lower().str.contains('pizza')) + (Pizzerias.nom_estab.str.lower().str.contains('pizzer')))
    Pizzerias=Pizzerias.iloc[a]
    return(Pizzerias)

def haversine(Olat,Olon, Dlat,Dlon):
    radius = 6371.  # km
    d_lat = np.radians(Dlat - Olat)
    d_lon = np.radians(Dlon - Olon)
    a = (np.sin(d_lat / 2.) * np.sin(d_lat / 2.) +
         np.cos(np.radians(Olat)) * np.cos(np.radians(Dlat)) *
         np.sin(d_lon / 2.) * np.sin(d_lon / 2.))
    c = 2. * np.arctan2(np.sqrt(a), np.sqrt(1. - a))
    d = radius * c
    return d



In [305]:
Tabla=pd.DataFrame([])
for i in range(0, len(dir)):
    Tabla=Tabla.append(extraer_pizzeria(dir[i]))

In [306]:
Tabla

,id,nom_estab,per_ocu,latitud,longitud,Manzana,Numero,Y_alta
4123,9291288,PIZZERIA PEPE S,0 a 5 personas,21.875509,-102.334044,ABELARDO RODRIGUEZ/ATENQUIQUE/ATEQUIZA/AHUALULCO,401.0,2020
5075,9269480,ABARROTES PIZZERIA Y PANADERIA,0 a 5 personas,32.514453,-116.730594,ALISO/CALLE DEL ÁLAMO/CALLE DEL CEREZO/TULE,5801.0,2020
58192,8231181,SUPER EXPRES Y PIZZERIA,0 a 5 personas,16.250192,-92.023385,PRIMERA ORIENTE SUR/TERCERA SUR ORIENTE/CUARTA...,NaN,2019
84572,554350,PLANET PIZZA,0 a 5 personas,31.652968,-106.452140,TENIENTE DANIEL GARCIA/OASIS DE ARMENIA/OASIS ...,NaN,2014
115756,8671918,PIZZAS PAPALUAPAN,0 a 5 personas,19.299150,-99.078415,RIO PAPALOAPAN/RIO GANGES/RIO NAZAS/DÉCIMA CER...,5.0,2019
...,...,...,...,...,...,...,...,...
180890,9361383,VENTA DE PIZZAS,0 a 5 personas,23.168789,-102.838851,JOSE VASCONCELOS/VALLE/MAYAS/MEXICAS,NaN,2021
180891,4618626,VENTA DE PIZZAS SIN NOMBRE,0 a 5 personas,23.074746,-102.721037,JUAN ALDAMA/CUAUHTEMOC/IGNACIO ALLENDE/MIGUEL ...,25.0,2010
181039,9103185,VIVA LA PIZZA,0 a 5 personas,22.951483,-102.705006,5 DE MAYO NORTE/PROFESOR JOSE REYES/VICENTE GU...,302.0,2019
181054,4601846,YARELY PIZZERIA,0 a 5 personas,21.864496,-102.912754,GUADALUPE VICTORIA/NINGUNO/NINGUNO/AQUILES SERDÁN,0.0,2010


In [307]:
def plot_locales(a,c,v,r):
    import plotly.express as px 
    fig=px.density_mapbox(c,lat='latitud',lon='longitud',hover_name='nom_estab'
    ,radius=6,center=dict (lat=np.mean(c.latitud.values),lon=np.mean(c.longitud.values)),
    
    zoom=r,mapbox_style='stamen-terrain')
    import plotly.graph_objects as go
    fig.add_trace(go.Scattermapbox(
        mode = "markers",
        lat=a.iloc[:]['latitud'].round(6).values.tolist(),
        lon=a.iloc[:]['longitud'].round(6).values.tolist(),
        marker = {'size': 13},text=a.iloc[:]['Dirección'].values.tolist()))
    fig.update_layout(mapbox_style="stamen-terrain", 
    mapbox_center_lat=np.mean(c.latitud.values),mapbox_center_lon=np.mean(c.longitud.values))
    fig.update_layout(margin=dict(b=0, t=0, l=0, r=0))
    fig.update_layout(showlegend=False)
    fig.update_layout(height=400,width=400)
    fig.update_coloraxes(showscale=False)
    fig.show()

In [308]:
plot_locales(cliente,Tabla,'Dirección',5)

In [309]:
Matriz_Distancias=pd.DataFrame([])
for c in range(0,len(cliente)):
    a=[]
    for i in range(0,len(Tabla)):
        a=a+[haversine(Tabla.iloc[i].latitud,Tabla.iloc[i].longitud,cliente.iloc[c].latitud,cliente.iloc[c].longitud)]
    Matriz_Distancias['cliente'+str(c)]=a
def competencia_cercana(cli,dist):    
    b='cliente'+str(cli)
    z=Matriz_Distancias.iloc[np.where(Matriz_Distancias[b].values<=dist)][b].index.values.tolist()
    
    return(Tabla.iloc[z])

In [310]:
competencia_cercana(3,3)

,id,nom_estab,per_ocu,latitud,longitud,Manzana,Numero,Y_alta
97288,9103897,3 SESENTA PIZZERIA DE BARRIO,6 a 10 personas,19.373831,-99.156132,MIGUEL LAURENT/TAJIN/PETEN/PILARES,1154.0,2019
97309,7232067,480 LA PIZZA,11 a 30 personas,19.353760,-99.158705,XICOTENCATL/GOMEZ FARIAS/MORELOS/MALINTZIN,256.0,2019
102112,974878,BAMBYS PIZZAS Y HAMBURGUESAS,6 a 10 personas,19.368841,-99.147555,7 SUR (MUNICIPIO LIBRE)/FILIPINAS/CANARIAS/PRE...,114.0,2010
102940,933559,BENEDETTIS PIZZA,6 a 10 personas,19.372900,-99.174533,7 SUR (FELIX CUEVAS)/MORAS/PATRICIO SANZ/SAN L...,326.0,2014
102942,870248,BENEDETTIS PIZZA,11 a 30 personas,19.398292,-99.170342,INSURGENTES SUR/MIGUEL ALEMÁN/XOLA/DIVISIÓN DE...,576.0,2010
...,...,...,...,...,...,...,...,...
144673,7232822,SALVAXE PIZZERIA ARTESANAL,6 a 10 personas,19.380726,-99.143883,ISABEL LA CATOLICA/NIGROMANTE/FRANCISCO FIELD ...,1147.0,2019
144691,8561457,SAN GIMIGNANO PIZZA Y PASTA,0 a 5 personas,19.353053,-99.157673,MALINTZIN/MORELOS/SAN PEDRO/CUAUHTÉMOC,24.0,2019
144722,933947,SANDWICH Y PIZZA CIELO,6 a 10 personas,19.364434,-99.187968,JOSE MARIA VELASCO/SAGREDO/REVOLUCION/REVOLUCIÓN,25.0,2014
152513,933117,TOMMY PIZZAS,11 a 30 personas,19.391069,-99.176986,NUEVA YORK/DAKOTA/ALABAMA/IDAHO,184.0,2014
